# Lesson 3: Building an Agent Reasoning Loop

In [ ]:
## Setup

from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [ ]:
import nest_asyncio
nest_asyncio.apply()

## Load the data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

## Setup the Query Tools

In [ ]:
## code for utils.py

# # TODO: abstract all of this into a function that takes in a PDF file name 

# from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex
# from llama_index.core.node_parser import SentenceSplitter
# from llama_index.core.tools import FunctionTool, QueryEngineTool
# from llama_index.core.vector_stores import MetadataFilters, FilterCondition
# from typing import List, Optional

# def get_doc_tools(
#     file_path: str,
#     name: str,
# ) -> str:
#     """Get vector query and summary query tools from a document."""

#     # load documents
#     documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
#     splitter = SentenceSplitter(chunk_size=1024)
#     nodes = splitter.get_nodes_from_documents(documents)
#     vector_index = VectorStoreIndex(nodes)
    
#     def vector_query(
#         query: str, 
#         page_numbers: Optional[List[str]] = None
#     ) -> str:
#         """Use to answer questions over the MetaGPT paper.
    
#         Useful if you have specific questions over the MetaGPT paper.
#         Always leave page_numbers as None UNLESS there is a specific page you want to search for.
    
#         Args:
#             query (str): the string query to be embedded.
#             page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE 
#                 if we want to perform a vector search
#                 over all pages. Otherwise, filter by the set of specified pages.
        
#         """
    
#         page_numbers = page_numbers or []
#         metadata_dicts = [
#             {"key": "page_label", "value": p} for p in page_numbers
#         ]
        
#         query_engine = vector_index.as_query_engine(
#             similarity_top_k=2,
#             filters=MetadataFilters.from_dicts(
#                 metadata_dicts,
#                 condition=FilterCondition.OR
#             )
#         )
#         response = query_engine.query(query)
#         return response
        
    
#     vector_query_tool = FunctionTool.from_defaults(
#         name=f"vector_tool_{name}",
#         fn=vector_query
#     )
    
#     summary_index = SummaryIndex(nodes)
#     summary_query_engine = summary_index.as_query_engine(
#         response_mode="tree_summarize",
#         use_async=True,
#     )
#     summary_tool = QueryEngineTool.from_defaults(
#         name=f"summary_tool_{name}",
#         query_engine=summary_query_engine,
#         description=(
#             "Use ONLY IF you want to get a holistic summary of MetaGPT. "
#             "Do NOT use if you have specific questions over MetaGPT."
#         ),
#     )

#     return vector_query_tool, summary_tool

In [ ]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

In [ ]:
## Setup Function Calling Agent

from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

print(response.source_nodes[0].get_content(metadata_mode="all"))

In [ ]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

response = agent.chat("Tell me the results over one of the above datasets.")


In [ ]:
## Lower-Level: Debuggability and Control

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)


In [ ]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

step_output = agent.run_step(task.task_id)


In [ ]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

In [ ]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

In [ ]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

step_output = agent.run_step(task.task_id)
print(step_output.is_last)

In [ ]:
response = agent.finalize_response(task.task_id)

print(str(response))